In [15]:
#!/usr/bin/env python3
#pip install  rioxarray==0.3.1
import pandas as pd
import xarray as xr
import os
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import matplotlib.colors
scriptsdir = os.getcwd()
from scipy.interpolate import griddata
from functools import reduce
import itertools
import argparse
import pickle


scenario = ["rcp26"]
time = [65]


sdms = ["GAM", "GBM"]
gcms = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
taxas = ["Mammals","Amphibians","Bird"]  # Add the second taxa here
#scenario = "rcp26"
#time=35
historical_time=1146

years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050',
         '2052', '2056', '2080', '2100', '2150', '2200', '2250']

def validate_values(data_array, tolerance=1e-6):
    """Check if values in the data array are NaN or between 0 and 1, and return invalid values."""
    # Use xarray's where method to filter out valid values (NaN or between 0 and 1)
    invalid_values = data_array.where((data_array < 0) | (data_array > 1.01), drop=True)

    # Check if there are any invalid values
    if invalid_values.size > 0:
        return False, invalid_values
    return True, None


# Create the dictionaries
newvalue_hist_sum = {}
newvalue_future_sum = {}
sumbin_hist_sum = {}
sumbin_future_sum = {}

# Initialize the dictionaries with SDM, GCM, and taxa keys
mean_newvalue_change = {}
mean_sum_bin_change = {}
mean_land_use_change = {}

for sdm in sdms:
    newvalue_hist_sum[sdm] = {}
    newvalue_future_sum[sdm] = {}
    sumbin_hist_sum[sdm] = {}
    sumbin_future_sum[sdm] = {}
    mean_newvalue_change[sdm] = {}
    mean_sum_bin_change[sdm] = {}
    mean_land_use_change[sdm] = {}

    for gcm in gcms:
        newvalue_hist_sum[sdm][gcm] = {}
        newvalue_future_sum[sdm][gcm] = {}
        sumbin_hist_sum[sdm][gcm] = {}
        sumbin_future_sum[sdm][gcm] = {}
        mean_newvalue_change[sdm][gcm] = {}
        mean_sum_bin_change[sdm][gcm] = {}
        mean_land_use_change[sdm][gcm] = {}

        for taxa in taxas:
            newvalue_hist_sum[sdm][gcm][taxa] = []
            newvalue_future_sum[sdm][gcm][taxa] = []
            sumbin_hist_sum[sdm][gcm][taxa] = []
            sumbin_future_sum[sdm][gcm][taxa] = []

# Loop over all taxa
for taxa in taxas:
    for sdm in sdms:
        dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + sdm + "/" + taxa + "/EWEMBI/"
        available_file = os.listdir(dir_species)
        available_names = [x.split("_[1146].nc")[0] for x in available_file]

        species_names = available_names
        # Define the netCDF file path
        netcdf_path_format_future = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/{}/{}/{}_[{}].nc"
        netcdf_path_format_hist = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/EWEMBI/{}_[{}].nc"

        # Loop over all species
        for species_name in species_names:
            # Loop over all models
            for gcm in gcms:

                # Open the netCDF files
                ds_newvalue_fut = xr.open_dataset(
                    netcdf_path_format_future.format(sdm, taxa, gcm, scenario[0], species_name, time[0]),
                    decode_times=False
                )
                ds_newvalue_hist = xr.open_dataset(
                    netcdf_path_format_hist.format(sdm, taxa, species_name, historical_time),
                    decode_times=False
                )

                # Get the newvalue and sum_bin
                newvalue_hist = ds_newvalue_hist["newvalue"]
                newvalue_future = ds_newvalue_fut["newvalue"]
                sum_bin_hist = ds_newvalue_hist["sum_bin"].isel(time=0)
                sum_bin_future = ds_newvalue_fut["sum_bin"].isel(time=0)

                # Validate the values and print invalid ones
                is_valid, invalid_values = validate_values(newvalue_hist)
                if not is_valid:
                    print(f"Invalid values found in newvalue_hist for {species_name} in {taxa}, {sdm}, {gcm}: {invalid_values}")

                is_valid, invalid_values = validate_values(newvalue_future)
                if not is_valid:
                    print(f"Invalid values found in newvalue_future for {species_name} in {taxa}, {sdm}, {gcm}: {invalid_values}")

                is_valid, invalid_values = validate_values(sum_bin_hist)
                if not is_valid:
                    print(f"Invalid values found in sum_bin_hist for {species_name} in {taxa}, {sdm}, {gcm}: {invalid_values}")

                is_valid, invalid_values = validate_values(sum_bin_future)
                if not is_valid:
                    print(f"Invalid values found in sum_bin_future for {species_name} in {taxa}, {sdm}, {gcm}: {invalid_values}")


Invalid values found in newvalue_future for Heteromys_anomalus in Mammals, GAM, IPSL-CM5A-LR: <xarray.DataArray 'newvalue' (lat: 1, lon: 1)>
array([[-1.73472348e-18]])
Coordinates:
  * lon      (lon) float64 -105.2
  * lat      (lat) float64 19.75
Invalid values found in sum_bin_future for Heteromys_anomalus in Mammals, GAM, IPSL-CM5A-LR: <xarray.DataArray 'sum_bin' (lat: 1, lon: 1)>
array([[-1.06448765e-18]])
Coordinates:
    time     float64 65.0
  * lon      (lon) float64 -105.2
  * lat      (lat) float64 19.75
Invalid values found in newvalue_future for Transandinomys_bolivaris in Mammals, GAM, IPSL-CM5A-LR: <xarray.DataArray 'newvalue' (lat: 1, lon: 1)>
array([[-2.77555756e-17]])
Coordinates:
  * lon      (lon) float64 -105.2
  * lat      (lat) float64 19.75
Invalid values found in sum_bin_future for Transandinomys_bolivaris in Mammals, GAM, IPSL-CM5A-LR: <xarray.DataArray 'sum_bin' (lat: 1, lon: 1)>
array([[-1.51285184e-17]])
Coordinates:
    time     float64 65.0
  * lon      (l